In [1]:
import numpy as np

In [2]:
def forward_conv_layer_adhoc(self, x, hconfig, pm):
  mb_size, xh, xw, xchn = x.shape
  kh, kw, _, ychn = pm['k'].shape

  conv = np.zeros((mb_size, xh, xw, ychn)) 
  bh, bw = (kh - 1)//2, (kw - 1)//2

  for n in range(mb_size):
    for r in range(xh):
      for c in range(xw):
        for m in range(ychn):
          #
          for i in range(kh):
            for j in range(kw):
              ii, jj = r + i - bh, c + j - bw
              if ii < 0 or ii >= xh: continue
              if jj < 0 or jj >= xw: continue 
              for k in range(xchn):
                curK = pm['k'][i][j][k][m]
                curX = x[n][ii][jj][k]
                conv[n][r][c][m] += curK * curX
  y = self.activate(conv + pm['b'], hconfig)
  return y, [x, y]

In [3]:
def forward_conv_layer_better(self, x, hconfig, pm):
  mb_size, xh, xw, xchn = x.shape
  kh, kw, _, ychn = pm['k'].shape

  bh, bw = (kh - 1)//2, (kw - 1)//2
  eh, ew = xh + kh - 1, xw + kw - 1

  conv = np.zeros((mb_size, xh, xw, ychn)) 
  x_temp = np.zeros((mb_size, eh, ew, xchn))
  x_temp[:, bh:bh+xh, bw:bw+xw, :] = x
  k_flat = pm['k'].transpose((3, 0, 1, 2)).reshape((ychn, -1))

  for n in range(mb_size):
    for r in range(xh):
      for c in range(xw):
        for m in range(ychn):
          x_flat = x_temp[n, r:r+kh, c:c+kw, :].flatten()
          conv[n, r, c, m] = (x_flat * k_flat[m]).sum()
    y = self.activate(conv + pm['b'], hconfig)
    return y, [x, y]

In [4]:
def get_ext_regions_for_conv(x, kh, kw):
  mb_size, xh, xw, xchn = x.shape
  return get_ext_regions(x, kh, kw, 0).reshape(mb_size * xh * xw, kh * kw * xchn)

def get_ext_regions(x, kh, kw, fill):
  mb_size, xh, xw, xchn = x.shape
  
  bh, bw = (kh - 1)//2, (kw - 1)//2
  eh, ew = xh + kh - 1, xw + kw - 1
  
  x_temp = np.zeros((mb_size, eh, ew, xchn), dtype="float32") + fill
  x_temp[:, bh:bh+xh, bw:bw+xw, :] = x
  ret = np.zeros((mb_size, xh, xw, kh, kw, xchn), dtype="float32")
  for i in range(xh):
    for j in range(xw):
      ret[:, i, j, :, :, :] = x_temp[:, i:i+kh, j:j+kw, :]
  return ret

In [5]:
def undo_ext_regions_for_conv(regs, x, kh, kw):
  mb_size, xh, xw, xchn = x.shape
  regs = regs.reshape((mb_size, xh, xw, kh, kw, xchn))
  return undo_ext_regions(regs, kh, kw)

def undo_ext_regions(regs, kh, kw):
  mb_size, xh, xw, kh, kw, xchn = regs.shape

  eh, ew = xh + kh - 1, xw + kw - 1
  bh, bw = (kh - 1)//2, (kw - 1)//2

  ret = np.zeros((mb_size, eh, ew, xchn), dtype="float")
  for r in range(xh):
    for c in range(xw):
      ret[:, r:r+kh, c:c+kw, :] += regs[:, r, c, :, :, :]
  return ret[:, bh:bh+xh, bw:bw+xw, :]

In [6]:
def get_ext_regions_for_conv2(x, kh, kw):
    mb_size, xh, xw, xchn = x.shape

    regs = get_ext_regions2(x, kh, kw, 0)
    regs = regs.transpose([2, 0, 1, 3, 4, 5])
    
    return regs.reshape([mb_size*xh*xw, kh*kw*xchn])

def get_ext_regions2(x, kh, kw, fill):
    mb_size, xh, xw, xchn = x.shape
    
    eh, ew = xh + kh - 1, xw + kw - 1
    bh, bw = (kh-1)//2, (kw-1)//2

    x_ext = np.zeros((mb_size, eh, ew, xchn), dtype = 'float32') + fill
    x_ext[:, bh:bh + xh, bw:bw + xw, :] = x
    
    regs = np.zeros((xh, xw, mb_size*kh*kw*xchn), dtype = 'float32')

    for r in range(xh):
        for c in range(xw):
            regs[r, c, :] = x_ext[:, r:r + kh, c:c + kw, :].flatten()

    return regs.reshape([xh, xw, mb_size, kh, kw, xchn])

In [7]:
def undo_ext_regions_for_conv2(regs, x, kh, kw):
    mb_size, xh, xw, xchn = x.shape

    regs = regs.reshape([mb_size, xh, xw, kh, kw, xchn])
    regs = regs.transpose([1, 2, 0, 3, 4, 5])
    
    return undo_ext_regions2(regs, kh, kw)

def undo_ext_regions2(regs, kh, kw):
    xh, xw, mb_size, kh, kw, xchn = regs.shape
    
    eh, ew = xh + kh - 1, xw + kw - 1
    bh, bw = (kh-1)//2, (kw-1)//2

    gx_ext = np.zeros([mb_size, eh, ew, xchn], dtype = 'float32')

    for r in range(xh):
        for c in range(xw):
            gx_ext[:, r:r + kh, c:c + kw, :] += regs[r, c]

    return gx_ext[:, bh:bh + xh, bw:bw + xw, :]

In [8]:
x = np.arange(16).reshape(2, 2, 2, 2)
x1 = get_ext_regions_for_conv(x, 2, 2)
x_back1 = undo_ext_regions_for_conv(x1, x, 2, 2)
print(x)
print(x_back1)

[[[[ 0  1]
   [ 2  3]]

  [[ 4  5]
   [ 6  7]]]


 [[[ 8  9]
   [10 11]]

  [[12 13]
   [14 15]]]]
[[[[ 0.  1.]
   [ 4.  6.]]

  [[ 8. 10.]
   [24. 28.]]]


 [[[ 8.  9.]
   [20. 22.]]

  [[24. 26.]
   [56. 60.]]]]


In [9]:
x = np.arange(3000).reshape(10, 10, 10, 3)
x1 = get_ext_regions_for_conv(x, 2, 2)
x2 = get_ext_regions_for_conv2(x, 2, 2)


x_back1 = undo_ext_regions_for_conv(x1, x, 2, 2)
print(np.sum(x != x_back1))

2970


In [10]:
x = np.arange(3000).reshape(10, 10, 10, 3)
x1 = get_ext_regions_for_conv(x, 3, 3)
x2 = get_ext_regions_for_conv2(x, 3, 3)

print(np.sum(x1 != x2))
print("===========================")
x_back1 = undo_ext_regions_for_conv(x1, x, 3, 3)
x_back2 = undo_ext_regions_for_conv(x1, x, 3, 3)
# print(np.sum(x_back1 != x_back2))
print(np.sum(x != x_back1))
# print(np.sum(x != x_back2))

0
2999


In [11]:
def cnn_basic_forward_conv_layer(self, x, hconfig, pm):
  mb_size, xh, xw, xchn = x.shape
  kh, kw, _, ychn = hconfig['k'].shape

  x_flat = get_ext_regions_for_conv(x, kh, kw)
  k_flat = pm['k'].reshape((kh * kw * xchn, ychn))
  conv_flat = np.matmul(x_flat, k_flat)
  conv = conv_flat.reshape((mb_size, xh, xw, ychn))

  y = self.activate(conv + pm['b'], hconfig)
  self.maps.append(y)
  return y, [x_flat, k_flat, x, y]

In [12]:
x = np.arange(16).reshape(2, 2, 2, 2)
print(x)

[[[[ 0  1]
   [ 2  3]]

  [[ 4  5]
   [ 6  7]]]


 [[[ 8  9]
   [10 11]]

  [[12 13]
   [14 15]]]]


In [14]:
avg = np.mean(x.reshape((-1, x.shape[-1])), axis=0)
print(avg)

[7. 8.]


In [15]:
x - avg

array([[[[-7., -7.],
         [-5., -5.]],

        [[-3., -3.],
         [-1., -1.]]],


       [[[ 1.,  1.],
         [ 3.,  3.]],

        [[ 5.,  5.],
         [ 7.,  7.]]]])

In [16]:
np.sum(x, axis=(0, 1, 2))

array([56, 64])